#### Import packages

In [ ]:
import time
import pandas as pd
import numpy as np
import random
import bisect
import pickle
import matplotlib.pyplot as plt  
from datetime import timedelta

#### Read in files

In [ ]:
medFilePath = r'C:\Users\momenzadeha\Documents\hypogly_pred\EHR_extracts\Meyer2306 All Meds.csv'
labFilePath = r'C:\Users\momenzadeha\Documents\hypogly_pred\EHR_extracts\meyer_2306_labs_with_order_time_2023_03_06.csv'
originalMedDf = pd.read_csv(medFilePath, parse_dates=['MAR_TAKEN_TIME'])
originalLabDf = pd.read_csv(labFilePath)
originalLabDf = originalLabDf[originalLabDf['PROC_NAME'].str.contains("POCT Glucose", na=False)]

#### Pre-processing

In [ ]:
def format_lab_df(originalLabDf):
    # Create a copy of the original dataframe to avoid modifying the input directly
    labDf = originalLabDf.copy()
    
    # Select only the relevant columns: 'PAT_ENC_CSN_ID', 'CONTACT_DATE', 'RESULT_TIME', and 'ORD_VALUE'
    labDf = labDf[['PAT_ENC_CSN_ID', 'CONTACT_DATE', 'RESULT_TIME', 'ORD_VALUE']]
    
    # Convert 'CONTACT_DATE' and 'RESULT_TIME' columns to datetime objects
    labDf[['CONTACT_DATE', 'RESULT_TIME']] = labDf[['CONTACT_DATE', 'RESULT_TIME']].apply(pd.to_datetime)
    
    # Replace various non-numeric values in the 'ORD_VALUE' column with numeric equivalents
    labDf['ORD_VALUE'].replace(['154bs', 'BS 175', 'bs120', '168 mg/dL', '159=bs', 'bs 120', '`185',
                                '250 mg/dL', 'BG 171 mg/dL', '250 mg/dL', '240 mg/dL', '130  mg/dL',
                                'BS 145', 'bs-134'],
                               ['154', '175', '120', '168', '159', '120', '185', '250', '171', '250', '240',
                                '130', '145', '134'], inplace=True)
    
    # Convert 'ORD_VALUE' column to numeric, setting invalid parsing as NaN (coerce errors)
    labDf['ORD_VALUE'] = pd.to_numeric(labDf['ORD_VALUE'], errors='coerce')
    
    # Drop rows where 'ORD_VALUE' has NaN values (i.e., non-numeric or missing)
    labDf = labDf.dropna(subset=['ORD_VALUE'])
    
    # Return the cleaned and formatted lab dataframe
    return labDf

In [ ]:
def format_med_df(originalMedDf):
    # Create a copy of the original medication dataframe to avoid modifying the input directly
    medDf = originalMedDf.copy()
    
    # Select only the relevant columns: 'MRN', 'CSN', 'MAR_TAKEN_TIME', 'MEDICATION_NM', and 'MAR_SIG'
    medDf = medDf[['MRN', 'CSN', 'MAR_TAKEN_TIME', 'MEDICATION_NM', 'MAR_SIG']]
    
    # Drop any rows that contain missing (NaN) values
    medDf = medDf.dropna()
    
    # Keep only the rows where 'MAR_SIG' (the medication dosage or signature) is greater than 0
    # This filters out rows where no medication was taken or an invalid dosage was recorded
    medDf = medDf[medDf['MAR_SIG'] > 0].reset_index(drop=True)
    
    # Return the cleaned and filtered medication dataframe
    return medDf

In [ ]:
labDf = format_lab_df(originalLabDf)
filteredLabDf = labDf.copy()

In [ ]:
filteredMedDf = format_med_df(originalMedDf)

##### filter out extraneous medication entries

In [ ]:
flush_list=['SODIUM CHLORIDE 0.9 % (FLUSH) IJ SYRG',
       '0.9% NACL IV LINE FLUSH BAG',
       'HEPARIN SOLUTION 2 UNITS/ML FOR FLUSH INTRAPROCEDURE (CATH LAB ONLY)',
       'HEPARIN, PORCINE (PF) 1000 UNIT/ML IJ SOLN (ADULT FLUSH)',
       'HEPARIN (PORCINE) IN NACL (PF) 1000 UNITS/500ML FOR ECMO FLUSH IV SOLP',
       'HEPARIN (PORCINE) IN NACL (PF) 1000 UNITS/500ML FOR ECMO FLUSH IV SOLP BOLUS FROM BAG',
       'ALBUMIN 5% IN LR FOR PORTAL FLUSH MIXTURE (INTRAPROCEDURE)',
       'HEPARIN FLUSH INFUSION FOR THROMBOLYSIS (5 UNITS/ML) IV SOLN',
       'EPINEPHRINE IN STERILE WATER (EPI FLUSH) INTRAPROCEDURE ',
       'HEPARIN PERIOTONEAL CATHETER FLUSH PEDIATRIC',
       'WASH SOLUTION (COVID-19 STUDY DRUG) 5% HUMAN SERUM ALBUMIN FOR IV LINE FLUSH',
       'SODIUM CHLORIDE 0.9% FOR ECMO FLUSH IV SOL BOLUS FROM BAG',
       'HEPARIN (PORCINE) IN NACL 1000 UNITS/500ML NS FOR  CONGENITAL HEART ECMO FLUSH FROM BAG',
       '0.9% NACL (EG-01-1962-03 STUDY DRUG) INTRAVENTRICULAR FLUSH- IRB #44821',
       'HEPARIN (PORCINE) IN NACL 500 UNITS/500ML NS FOR  CONGENITAL HEART ECMO FLUSH FROM BAG',
       'SODIUM CHLORIDE 0.9% FOR ECMO FLUSH',
       'ALBUMIN 25% IN LR FOR PORTAL FLUSH MIXTURE (INTRAPROCEDURE)',
       'HEPARIN LOCK FLUSH (PORCINE)  10 UNITS/ML IV SOLN',
       'SODIUM CHLORIDE 0.9 % IV SOLP']
irrig_list=['NEOMYCIN-POLYMYXIN-BACITRACIN IRRIGATION',
       'WATER FOR IRRIGATION, STERILE IR SOLN',
       'CEFAZOLIN (ANCEF)-VANCO-GENT-CLINDAMYCIN-BACITRACIN-TOBRA IRRIGATION MIXTURE (INTRAPROCEDURE)',
       'LACTATED RINGERS IRRIGATION INTRAPROCEDURE',
       'PAPAVERINE IN NS FOR IRRIGATION (FOR CARDIAC) MIXTURE (INTRAPROCEDURE)',
       'SIMETHICONE IN STERILE WATER FOR IRRIGATION MIXTURE (INTRAPROCEDURE)',
       'BACITRACIN IN NS FOR IRRIGATION MIXTURE (INTRAPROCEDURE)',
       'VANCOMYCIN 500 MG IN 200 ML 0.9% NACL POCKET IRRIGATION FOR INTRAPROCEDURE',
       'GENTAMICIN 80 MG IN 200 ML 0.9% NACL POCKET IRRIGATION FOR INTRAPROCEDURE',
       'AMPHOTERICIN B IRRIGATION',
       'NEOMYCIN-POLYMYXIN B (NEOSPORIN) IRRIGATION SOLUTION MIXTURE (INTRAPROCEDURE)',
       'BETADINE 10%, NS IRRIGATION MIXTURE (INTRAPROCEDURE)',
       'POLYMYXIN B IRRIGATION SOLUTION (INTRAPROCEDURE)',
       'AMPHOTERICIN B (FUNGIZONE) IRRIGATION MIXTURE (INTRAPROCEDURE)',
       'PAPAVERINE, VERAPAMIL IN NS FOR IRRIGATION (FOR CARDIAC) INTRAPROCEDE',
       'EPINEPHRINE IN NORMAL SALINE FOR IRRIGATION MIXTURE (INTRAPROCEDURE)',
       'GENTAMICIN IN NS IRRIGATION INTRAPROCEDURE ONLY',
       'NEOSPORIN-POLYMYXIN B IRRIGATION SOLUTION',
       'IRRIGATION SOLNS PHYSIOLOGICAL IR SOLN',
       'CARDIOPLEGIC SOLUTION WITH GLUTAMATE/ASPARTATE IRRIGATION',
       'ALUM 10 GRAM IN STERILE WATER 1,000 ML IRRIGATION',
       'BACTROBAN NASAL IRRIGATION',
       'BALANCED SALTS FOR INTRAOCULAR IRRIGATION MIXTURE (INTRAPROCEDURE)','SODIUM CHLORIDE  0.9 % IR SOLN']
iohexol=['IOHEXOL 300 MG IODINE/ML IV SOLN (RADIANT USE)','IOHEXOL 350 MG IODINE/ML IV SOLN']
non_meds=['DRUG LEVEL REMINDER', 'HELP', 'IMS TEMPLATE', 'IMPELLA PURGE SOLUTION BUILDER', 'MMX ORAL SUSPENSION (IP ORDERING ONLY)']

In [ ]:
# Remove rows where the medication name is irrigation, flush, non-med, or iohexol
filteredMedDf_clean = filteredMedDf[~filteredMedDf['MEDICATION_NM'].isin(irrig_list+flush_list+non_meds+iohexol)]

##### Select top 500 medications + any other anti-hyperglycemics not in top 500 list

In [ ]:
replace_dict = {
    'INSULIN LISPRO 100 UNITS/ML SC SOLN': 'INSULIN LISPRO 100 UNITS/ML SOLN',
    'INSULIN LISPRO 100 UNITS/ML SC SOLN (SLIDING SCALE)': 'INSULIN LISPRO 100 UNITS/ML SOLN',
    'INSULIN REGULAR HUMAN 100 UNITS/ML IJ SOLN (SLIDING SCALE)': 'INSULIN REGULAR HUMAN 100 UNITS/ML IJ SOLN',
    'INSULIN REGULAR IN 0.9 % NACL 100 UNIT/100 ML (1 UNIT/ML) IV SOLN': 'INSULIN REGULAR IN 0.9% NACL 100 UNITS/100 ML (ADULT)',
    'INSULIN ASPART U-100 100 UNITS/ML SC SOLN (SLIDING SCALE)': 'INSULIN ASPART U-100 100 UNITS/ML SC SOLN',
    'INSULIN REGULAR HUMAN 100 UNIT/ML IJ SOLN (PRL 1772 & 2600  ONLY)': 'INSULIN REGULAR HUMAN 100 UNITS/ML IJ SOLN',
    'INSULIN GLARGINE 100 UNITS/ML SC SOLN': 'INSULIN GLARGINE 100 UNITS/ML',
    'INSULIN GLARGINE 100 UNITS/ML SC INPN': 'INSULIN GLARGINE 100 UNITS/ML',
    'INSULIN NPH AND REGULAR HUMAN 100 UNIT/ML (70-30) SC SUSP': 'INSULIN NPH AND REGULAR HUMAN 100 UNIT/ML (70-30) SC'
}

filteredMedDf_clean = filteredMedDf_clean.replace(replace_dict)

In [ ]:
target_med_list=['INSULIN LISPRO 100 UNITS/ML SOLN',
                 'INSULIN REGULAR HUMAN 100 UNITS/ML IJ SOLN',
                 'INSULIN REGULAR IN 0.9% NACL 100 UNITS/100 ML (ADULT)',
                 'INSULIN ASPART U-100 100 UNITS/ML SC SOLN',
                 'INSULIN GLARGINE 100 UNITS/ML',
                 'INSULIN NPH AND REGULAR HUMAN 100 UNIT/ML (70-30) SC',
                 'SITAGLIPTIN 100 MG PO TABS', 'SITAGLIPTIN  50 MG PO TABS','SITAGLIPTIN  25 MG PO TABS',
                 'GLIMEPIRIDE 4 MG PO TABS', 'GLIMEPIRIDE 1 MG PO TABS','GLIMEPIRIDE 2 MG PO TABS',
                 'GLIPIZIDE 10 MG PO TABS', 'GLIPIZIDE  5 MG PO TABS',
                 'GLIPIZIDE  5 MG PO TR24','GLIPIZIDE 10 MG PO TR24',
                 'GLYBURIDE 1.25 MG PO TABS','GLYBURIDE 5 MG PO TABS',
                 'PIOGLITAZONE 30 MG PO TABS', 'PIOGLITAZONE 15 MG PO TABS','PIOGLITAZONE 45 MG PO TABS',
                 'CSHS PARENTERAL NUTRITITION ORDER ADULT','CSHS PERIPHERAL PARENTERAL NUTRITION ADULT','CSHS PARENTERAL NUTRITION ORDER CLINIMIX E MIXTURE']     

In [ ]:
topNum = 500
top500_list=filteredMedDf_clean['MEDICATION_NM'].value_counts()[:topNum].index.to_list()
top500_DM_meds=target_med_list+top500_list
targfilteredMedDf=filteredMedDf_clean[filteredMedDf_clean['MEDICATION_NM'].isin(top500_DM_meds)]

#### Merge dataframes - medications leading up to labs

In [ ]:
csnToResultTimesDict = filteredLabDf.sort_values('RESULT_TIME').groupby(
        'PAT_ENC_CSN_ID')['RESULT_TIME'].agg(list).to_dict()
csnResultTimeToGlycemicLevel = filteredLabDf.set_index(['PAT_ENC_CSN_ID', 'RESULT_TIME'])['ORD_VALUE'].to_dict()
medsBeforeLabEventDf = targfilteredMedDf[targfilteredMedDf['CSN'].isin(set(csnToResultTimesDict.keys()))].dropna()
medsBeforeLabEventDf = medsBeforeLabEventDf.sort_values(['MAR_TAKEN_TIME']).reset_index(drop=True)

#### Calculate time differences between medication administration and lab results

In [ ]:
# Sort the DataFrame by 'RESULT_TIME' within each 'PAT_ENC_CSN_ID' group
filteredLabDf.sort_values(by=['PAT_ENC_CSN_ID', 'RESULT_TIME'], inplace=True)

# Initialize an empty list to store the selected rows
selected_rows = []
counter = 0

# Group by 'PAT_ENC_CSN_ID' and iterate over each group of lab records
for _, group in filteredLabDf.groupby('PAT_ENC_CSN_ID'):
    # Skip groups with fewer than 2 records (no valid comparisons can be made)
    if len(group) < 2:
        continue  
    
    # Randomly select one 'RESULT_TIME' from the group
    random_time = random.randint(0, len(group)-1)
    random_measure = group['RESULT_TIME'].tolist()[random_time]
    
    # Define a time window: between 4 and 16 hours prior to the randomly selected 'RESULT_TIME'
    tminus4 = random_measure - timedelta(hours=4)
    tminus16 = random_measure - timedelta(hours=16)

    # Filter for rows where 'RESULT_TIME' is within the 4 to 16 hours window and 'ORD_VALUE' is non-zero
    valid_rows = group[(group['RESULT_TIME'].between(tminus16, tminus4)) & (group['ORD_VALUE'] != 0)]

    # If no valid rows are found, try up to 5 times to select a new random 'RESULT_TIME'
    if valid_rows.empty:
        i = 0
        while i < 5:
            # Randomly select another 'RESULT_TIME' from the group
            random_time = random.randint(0, len(group)-1)
            random_measure = group['RESULT_TIME'].tolist()[random_time]
            
            # Recalculate the 4 to 16 hour window
            tminus4 = random_measure - timedelta(hours=4)
            tminus16 = random_measure - timedelta(hours=16)
            
            # Filter for valid rows again
            valid_rows = group[(group['RESULT_TIME'].between(tminus16, tminus4)) & (group['ORD_VALUE'] != 0)]
            
            # If valid rows are found, exit the loop
            if not valid_rows.empty:
                break
            i += 1

    # If valid rows were found, select the row corresponding to the original random 'RESULT_TIME'
    if not valid_rows.empty:
        selected_row = group.iloc[random_time, :]  # Select the row at the random index
        selected_rows.append(selected_row)  # Add the selected row to the list of selected rows
    
    counter += 1
    
    # Print progress every 1000 iterations
    if counter % 1000 == 0:
        print(counter)

# Create a DataFrame from the selected rows
selected_df = pd.DataFrame(selected_rows)

In [ ]:
# Select and reset the index of the columns 'PAT_ENC_CSN_ID' and 'RESULT_TIME' from selected_df
rand_BG_df = selected_df[['PAT_ENC_CSN_ID', 'RESULT_TIME']].reset_index(drop=True)

# Rename 'PAT_ENC_CSN_ID' column to 'CSN' for consistency in merging
rand_BG_df.rename(columns={'PAT_ENC_CSN_ID': 'CSN'}, inplace=True)

# Merge the medication data (medsBeforeLabEventDf) with the randomized lab data (rand_BG_df) on 'CSN'
merge = pd.merge(medsBeforeLabEventDf, rand_BG_df, how='inner', on='CSN')

In [ ]:
with open(r'/Users/momenzadeha/Documents/hypogly_pred/MELRoutputCSNs.pkl', 'wb') as handle:
    pickle.dump(rand_BG_df, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(r'/Users/momenzadeha/Documents/hypogly_pred/outputBG_df.pkl', 'wb') as handle:
    pickle.dump(selected_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### Calculate differences between medication and random BG time

In [ ]:
def create_time_difference_column_in_meds_before_lab_event_df(medsBeforeLabEventDf):
    diffFromLabColumn = medsBeforeLabEventDf['RESULT_TIME'] - medsBeforeLabEventDf['MAR_TAKEN_TIME']
    diffFromLabColumn = [np.timedelta64(x, 'm') for x in diffFromLabColumn]
    medsBeforeLabEventDf['TIME_DIFF'] = diffFromLabColumn
    medsBeforeLabEventDf = medsBeforeLabEventDf[
        ['CSN', 'MAR_TAKEN_TIME', 'RESULT_TIME', 'TIME_DIFF', 'MEDICATION_NM', 'MAR_SIG']]
    return medsBeforeLabEventDf

In [ ]:
medsBeforeLabEventDf_diff = create_time_difference_column_in_meds_before_lab_event_df(merge)

#### Filter for medications only in the lookback window

In [ ]:
def establish_random_delay_before_med_for_each_csn(csnRts, delayTimeLength):
    # Create a dictionary where each 'CSN' (unique patient encounter) from 'csnRts'
    # is assigned the same 'delayTimeLength'
    return {key: delayTimeLength for key in csnRts}

In [ ]:
def divide_times_into_segments_for_each_csn_result_time(csnRt, timeDiffs, randomDelayDict, numSections, timeWindowHours):
    # Calculate the endTime as a timedelta (in minutes) based on the random delay for the given 'CSN'
    endTime = np.timedelta64(int(np.round(randomDelayDict[csnRt] * 60)), 'm')
    
    # Calculate the startTime by adding a modifiable 'segmentLengthHours' value to the endTime
    startTime = endTime + np.timedelta64(timeWindowHours * 60, 'm')  # 'segmentLengthHours' modifiable

    # Generate evenly spaced segment cutoffs from endTime to startTime
    segmentCutoffs = np.linspace(endTime.astype(int), startTime.astype(int), numSections + 1)
    
    # Convert each segment cutoff into a timedelta64 in minutes
    segmentCutoffs = [np.timedelta64(int(x), 'm') for x in segmentCutoffs]
    
    # Convert the 'timeDiffs' list into timedelta64 in minutes
    timeDiffs = [np.timedelta64(x, 'm') for x in timeDiffs]
    
    # Categorize each time difference into one of the segments
    segmentCategories = np.searchsorted(segmentCutoffs, timeDiffs, side='right')    
    return segmentCategories

In [ ]:
def identify_time_section_from_med_to_event(medsBeforeLabEventDf, numSections, randomDelayMaxLength, timeWindowHours):
    # Create a copy of the dataframe to avoid modifying the original data
    medsBeforeLabEventDf = medsBeforeLabEventDf.copy()

    # Sort the dataframe by 'CSN', 'RESULT_TIME', and 'TIME_DIFF' to ensure proper grouping
    medsBeforeLabEventDf = medsBeforeLabEventDf.sort_values(['CSN', 'RESULT_TIME', 'TIME_DIFF']).reset_index(drop=True)

    # Establish a random delay for each CSN-RESULT_TIME combination, using the maximum random delay length
    randomDelayDict = establish_random_delay_before_med_for_each_csn(set(zip(medsBeforeLabEventDf['CSN'], medsBeforeLabEventDf['RESULT_TIME'])), randomDelayMaxLength)

    # Apply the 'divide_times_into_segments_for_each_csn_result_time' function to create 'SEGMENT' categories for each CSN group
    medsBeforeLabEventDf['SEGMENT'] = medsBeforeLabEventDf.groupby(['CSN', 'RESULT_TIME'])['TIME_DIFF'].transform(
        lambda timeDiffs: divide_times_into_segments_for_each_csn_result_time(timeDiffs.name, timeDiffs, randomDelayDict, numSections, timeWindowHours)
    )

    # Return the modified dataframe and the random delay dictionary
    return medsBeforeLabEventDf, randomDelayDict

In [ ]:
# Define the number of segments, random delay maximum length, and the time window in hours
numSegments = 1  # Only 1 segment to divide the time period into
randomDelayMaxLength = 4  # Maximum random delay (e.g., 4 hours) for medications before the lab event
timeWindowHours = 24  # Time window (24 hours) to be used in the analysis

# Set a random seed for reproducibility when generating random delays
random.seed(0)

# Call the function to identify the time segment for each medication event, based on the time differences
# and the random delay assigned to each CSN (patient encounter)
medsBeforeLabEventDf_seg, randomDelayDict = identify_time_section_from_med_to_event(
    medsBeforeLabEventDf_diff, numSegments, randomDelayMaxLength, timeWindowHours)

In [ ]:
# Filter the DataFrame to exclude rows where the segment is 0 or 2
medsBeforeLabEventDf_seg_red = medsBeforeLabEventDf_seg[
    (medsBeforeLabEventDf_seg['SEGMENT'] != 0) & (medsBeforeLabEventDf_seg['SEGMENT'] != 2)
]

# Group by 'CSN' and 'MEDICATION_NM', aggregating the 'MAR_SIG' (medication dosage) by summing it
medsBeforeLabEventDf_seg_red_agg = medsBeforeLabEventDf_seg_red.groupby(['CSN', 'MEDICATION_NM'])['MAR_SIG'].sum()

# Create a pivot table with 'CSN' as the index and 'MEDICATION_NM' as columns, summing 'MAR_SIG'
medsBeforeLabEventDf_seg_red_agg_piv = medsBeforeLabEventDf_seg_red_agg.unstack().reset_index()

# Final DataFrame: 'CSN' is the index, with 'MAR_SIG' summed across different medications as columns
medsBeforeLabEventDf_seg_red_agg_piv = medsBeforeLabEventDf_seg_red_agg_piv.fillna(0) 

In [ ]:
with open(r'/Users/momenzadeha/Documents/hypogly_pred/top500_DM_meds_12h.pkl', 'wb') as handle:
    pickle.dump(medsBeforeLabEventDf_seg_red_agg_piv, handle, protocol=pickle.HIGHEST_PROTOCOL)